# WATER SCARCITY by country (rareté de l'eau)
## with prediction of future situation
## Target (objectif de prediction) : 
##    accessible drinking water per inhabitant

## 1. Scrapping

- Starting Scrapping
- Managing connexion shutdown, i.e. restarting loading at the same point
- Random time scrapping in order to stay hidden
- Manage 'my user agent' in order to scrap in 'browser mode' and not 'robot mode'
- Sites à scrapper :
- ---Consommation moyenne d'eau par habitant et par pays
- - https://www.cieau.com/le-metier-de-leau/ressource-en-eau-eau-potable-eaux-usees/la-consommation-deau-domestique-est-elle-la-meme-a-travers-le-monde/
- - https://fr.wikipedia.org/wiki/Consommation_d%27eau_domestique_par_pays
- --- Consommation d'eau virtuelle
- - https://www.consoglobe.com/eau-virtuelle-conflit-3120-cg
- - https://www.consoglobe.com/eau-virtuelle-consommation-cachee-cg
The idea with scrapping  is :
- each scrapping is specific to one web site (html code for balises)
- if web site is javascript encoded, you have to use a javascript interface to a 'false internet explorer (google or other)' , then send the 'html transformation' to BeautifulSoup

## 2. Ressources
FAO and UNO sites are web accessing datas, directly or with 'wget' command
- FAO http://www.fao.org/nr/water/aquastat/data/query/index.html
- UNO https://population.un.org/wpp/Download/Standard/Population/
- Population Mondiale par Pays ou Région (UNO)
- Total Population per Country (FAO Aquastat)
- Water disponibility per Country or Region (FAO Aquastat)
- Water consumption per inhabitant, by Country or Region (FAO + divers documents moyennés)
- [GDP per capita] = [Gross Domestic Product (GDP)]/[Total population]/1000  (FAO Aquastat)

In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 8000)
#pd.set_option('display.width', 1000)

In [5]:
dataf = pd.read_csv("../data/FAO_aquastat_flat_1958_2017.csv")
datac = pd.read_csv("../data/FAO_aquastat_tables_1958_2017.csv")

In [6]:
dataf.head()

,Area,Area Id,Variable Name,Variable Id,Year,Value,Symbol,Md
Austria,11.0,Total population,4104.0,1962.0,7157.0,E,NaN,NaN
Austria,11.0,Total population,4104.0,1967.0,7397.0,E,NaN,NaN
Austria,11.0,Total population,4104.0,1972.0,7581.0,E,NaN,NaN
Austria,11.0,Total population,4104.0,1977.0,7637.0,E,NaN,NaN
Austria,11.0,Total population,4104.0,1982.0,7602.0,E,NaN,NaN


In [7]:
dataf1 = dataf.copy()
dataf1 = dataf1.reset_index()
dataf1 = dataf1.rename(columns = {'index':'country', 
                                  'Area Id':'attribute',
                                  'Variable Id' : 'year',
                                  'Year' : 'value'})
dataf1.drop(columns=['Area','Variable Name','Value','Symbol','Md'], inplace=True)
dataf1.head()

,country,attribute,year,value
0,Austria,Total population,1962.0,7157.0
1,Austria,Total population,1967.0,7397.0
2,Austria,Total population,1972.0,7581.0
3,Austria,Total population,1977.0,7637.0
4,Austria,Total population,1982.0,7602.0


In [8]:
print(dataf1.info())
print(dataf1.iloc[7070])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7140 entries, 0 to 7139
Data columns (total 4 columns):
country      7139 non-null object
attribute    7071 non-null object
year         7071 non-null float64
value        7071 non-null float64
dtypes: float64(2), object(2)
memory usage: 223.2+ KB
None
country             United States of America
attribute    Environmental Flow Requirements
year                                    2017
value                                   1491
Name: 7070, dtype: object


In [9]:
datac.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1958-1962,Unnamed: 4,Unnamed: 5,Unnamed: 6,1963-1967,Unnamed: 8,Unnamed: 9,Unnamed: 10,1968-1972,Unnamed: 12,Unnamed: 13,Unnamed: 14,1973-1977,Unnamed: 16,Unnamed: 17,Unnamed: 18,1978-1982,Unnamed: 20,Unnamed: 21,Unnamed: 22,1983-1987,Unnamed: 24,Unnamed: 25,Unnamed: 26,1988-1992,Unnamed: 28,Unnamed: 29,Unnamed: 30,1993-1997,Unnamed: 32,Unnamed: 33,Unnamed: 34,1998-2002,Unnamed: 36,Unnamed: 37,Unnamed: 38,2003-2007,Unnamed: 40,Unnamed: 41,Unnamed: 42,2008-2012,Unnamed: 44,Unnamed: 45,Unnamed: 46,2013-2017,Unnamed: 48,Unnamed: 49
0,Austria,Total population (1000 inhab),1962.0,7157.0,E,NaN,1967.0,7397.0,E,NaN,1972.0,7581.0,E,NaN,1977.0,7637.0,E,NaN,1982.0,7602.0,E,NaN,1987.0,7639.0,E,NaN,1992.0,7831.0,E,NaN,1997.0,8033.0,E,NaN,2002.0,8134.0,E,NaN,2007.0,8312.0,E,NaN,2012.0,8518.0,E,NaN,2017.0,8735.0,E,NaN
1,Austria,Rural population (1000 inhab),1962.0,2527.0,E,NaN,1967.0,2596.0,E,NaN,1972.0,2644.0,E,NaN,1977.0,2667.0,E,NaN,1982.0,2660.0,E,NaN,1987.0,2655.0,E,NaN,1992.0,2700.0,E,NaN,1997.0,2760.0,E,NaN,2002.0,2809.0,E,NaN,2007.0,2840.0,E,NaN,2012.0,2943.0,E,NaN,2017.0,3036.0,E,NaN
2,Austria,Urban population (1000 inhab),1962.0,4630.0,E,NaN,1967.0,4801.0,E,NaN,1972.0,4937.0,E,NaN,1977.0,4970.0,E,NaN,1982.0,4942.0,E,NaN,1987.0,4984.0,E,NaN,1992.0,5131.0,E,NaN,1997.0,5273.0,E,NaN,2002.0,5325.0,E,NaN,2007.0,5472.0,E,NaN,2012.0,5575.0,E,NaN,2017.0,5699.0,E,NaN
3,Austria,Population density (inhab/km2),1962.0,85.32,E,NaN,1967.0,88.19,E,NaN,1972.0,90.38,E,NaN,1977.0,91.05,E,NaN,1982.0,90.63,E,NaN,1987.0,91.07,E,NaN,1992.0,93.36,E,NaN,1997.0,95.77,E,NaN,2002.0,96.97,E,NaN,2007.0,99.09,E,NaN,2012.0,101.5,E,NaN,2017.0,104.1,E,NaN
4,Austria,Gross Domestic Product (GDP) (current US$),1962.0,7.75611021E9,E,NaN,1967.0,1.1579431669E10,E,NaN,1972.0,2.2006470861E10,E,NaN,1977.0,5.142158563E10,E,NaN,1982.0,7.1103585384E10,E,NaN,1987.0,1.24E11,E,NaN,1992.0,1.95E11,E,NaN,1997.0,2.12E11,E,NaN,2002.0,2.13E11,E,NaN,2007.0,3.86E11,E,NaN,2012.0,4.07E11,E,NaN,2015.0,3.74E11,E,NaN


In [10]:
datac = datac.rename(columns = {'Unnamed: 0':'country', 
                                'Unnamed: 1':'attribute',
                                'Unnamed: 2':'year1',
                                'Unnamed: 6':'year2',
                                'Unnamed: 10':'year3',
                                'Unnamed: 14':'year4',
                                'Unnamed: 18':'year5',
                                'Unnamed: 22':'year6',
                                'Unnamed: 26':'year7',
                                'Unnamed: 30':'year8',                               
                                'Unnamed: 34':'year9',
                                'Unnamed: 38':'year10',
                                'Unnamed: 42':'year11',
                                'Unnamed: 46':'year12'
                               })
datac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 50 columns):
country        905 non-null object
attribute      837 non-null object
year1          462 non-null float64
1958-1962      492 non-null object
Unnamed: 4     337 non-null object
Unnamed: 5     5 non-null object
year2          462 non-null float64
1963-1967      492 non-null object
Unnamed: 8     337 non-null object
Unnamed: 9     5 non-null object
year3          503 non-null float64
1968-1972      533 non-null object
Unnamed: 12    368 non-null object
Unnamed: 13    5 non-null object
year4          505 non-null float64
1973-1977      535 non-null object
Unnamed: 16    375 non-null object
Unnamed: 17    5 non-null object
year5          535 non-null float64
1978-1982      565 non-null object
Unnamed: 20    397 non-null object
Unnamed: 21    5 non-null object
year6          535 non-null float64
1983-1987      565 non-null object
Unnamed: 24    399 non-null object
Unnamed: 25    5 non-nul

In [11]:
datac.columns

Index(['country', 'attribute', 'year1', '1958-1962', 'Unnamed: 4',
       'Unnamed: 5', 'year2', '1963-1967', 'Unnamed: 8', 'Unnamed: 9', 'year3',
       '1968-1972', 'Unnamed: 12', 'Unnamed: 13', 'year4', '1973-1977',
       'Unnamed: 16', 'Unnamed: 17', 'year5', '1978-1982', 'Unnamed: 20',
       'Unnamed: 21', 'year6', '1983-1987', 'Unnamed: 24', 'Unnamed: 25',
       'year7', '1988-1992', 'Unnamed: 28', 'Unnamed: 29', 'year8',
       '1993-1997', 'Unnamed: 32', 'Unnamed: 33', 'year9', '1998-2002',
       'Unnamed: 36', 'Unnamed: 37', 'year10', '2003-2007', 'Unnamed: 40',
       'Unnamed: 41', 'year11', '2008-2012', 'Unnamed: 44', 'Unnamed: 45',
       'year12', '2013-2017', 'Unnamed: 48', 'Unnamed: 49'],
      dtype='object')

In [12]:
datac1 = datac.copy()
for v in range(50) :
    try :
        datac1.drop(columns=f'Unnamed: {v}', inplace=True)
    except : 
        pass

In [13]:
datac1.columns

Index(['country', 'attribute', 'year1', '1958-1962', 'year2', '1963-1967',
       'year3', '1968-1972', 'year4', '1973-1977', 'year5', '1978-1982',
       'year6', '1983-1987', 'year7', '1988-1992', 'year8', '1993-1997',
       'year9', '1998-2002', 'year10', '2003-2007', 'year11', '2008-2012',
       'year12', '2013-2017'],
      dtype='object')

In [28]:
datac1.head(10)

,country,attribute,year1,1958-1962,year2,1963-1967,year3,1968-1972,year4,1973-1977,year5,1978-1982,year6,1983-1987,year7,1988-1992,year8,1993-1997,year9,1998-2002,year10,2003-2007,year11,2008-2012,year12,2013-2017
0,Austria,Total population (1000 inhab),1962.0,7157.0,1967.0,7397.0,1972.0,7581.0,1977.0,7637.0,1982.0,7602.0,1987.0,7639.0,1992.0,7831.0,1997.0,8033.0,2002.0,8134.0,2007.0,8312.0,2012.0,8518.0,2017.0,8735.0
1,Austria,Rural population (1000 inhab),1962.0,2527.0,1967.0,2596.0,1972.0,2644.0,1977.0,2667.0,1982.0,2660.0,1987.0,2655.0,1992.0,2700.0,1997.0,2760.0,2002.0,2809.0,2007.0,2840.0,2012.0,2943.0,2017.0,3036.0
2,Austria,Urban population (1000 inhab),1962.0,4630.0,1967.0,4801.0,1972.0,4937.0,1977.0,4970.0,1982.0,4942.0,1987.0,4984.0,1992.0,5131.0,1997.0,5273.0,2002.0,5325.0,2007.0,5472.0,2012.0,5575.0,2017.0,5699.0
3,Austria,Population density (inhab/km2),1962.0,85.32,1967.0,88.19,1972.0,90.38,1977.0,91.05,1982.0,90.63,1987.0,91.07,1992.0,93.36,1997.0,95.77,2002.0,96.97,2007.0,99.09,2012.0,101.5,2017.0,104.1
4,Austria,Gross Domestic Product (GDP) (current US$),1962.0,7.75611021E9,1967.0,1.1579431669E10,1972.0,2.2006470861E10,1977.0,5.142158563E10,1982.0,7.1103585384E10,1987.0,1.24E11,1992.0,1.95E11,1997.0,2.12E11,2002.0,2.13E11,2007.0,3.86E11,2012.0,4.07E11,2015.0,3.74E11
5,Austria,GDP per capita (current US$/inhab),1962.0,1084.0,1967.0,1565.0,1972.0,2903.0,1977.0,6733.0,1982.0,9353.0,1987.0,16232.0,1992.0,24901.0,1997.0,26391.0,2002.0,26186.0,2007.0,46439.0,2012.0,47781.0,2015.0,42816.0
6,Austria,Human Development Index (HDI) [highest = 1] (-),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990.0,0.7935,NaN,NaN,2000.0,0.8364,NaN,NaN,2012.0,0.8839,2014.0,0.885
7,Austria,Long-term average annual precipitation in dept...,1962.0,1110.0,1967.0,1110.0,1972.0,1110.0,1977.0,1110.0,1982.0,1110.0,1987.0,1110.0,1992.0,1110.0,1997.0,1110.0,2002.0,1110.0,2007.0,1110.0,2012.0,1110.0,2017.0,1110.0
8,Austria,Long-term average annual precipitation in volu...,1962.0,93.11,1967.0,93.11,1972.0,93.11,1977.0,93.11,1982.0,93.11,1987.0,93.11,1992.0,93.11,1997.0,93.11,2002.0,93.11,2007.0,93.11,2012.0,93.11,2017.0,93.11
9,Austria,Surface water produced internally (10^9 m3/year),1962.0,55.0,1967.0,55.0,1972.0,55.0,1977.0,55.0,1982.0,55.0,1987.0,55.0,1992.0,55.0,1997.0,55.0,2002.0,55.0,2007.0,55.0,2012.0,55.0,2017.0,55.0


In [15]:
print('dataf1 nb rows and columns')
print(dataf1.shape[0],dataf1.shape[1])
print('datac1 nb rows and columns')
print(datac1.shape[0],datac1.shape[1])

dataf1 nb rows and columns
7140 4
datac1 nb rows and columns
935 26


In [16]:
dataf1.iloc[7070] # C'est le dernier champ intéressant, après ce sont les données du fichier

country             United States of America
attribute    Environmental Flow Requirements
year                                    2017
value                                   1491
Name: 7070, dtype: object

In [17]:
dataf2 = dataf1[dataf1.index < 7071]
dataf2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7071 entries, 0 to 7070
Data columns (total 4 columns):
country      7071 non-null object
attribute    7071 non-null object
year         7071 non-null float64
value        7071 non-null float64
dtypes: float64(2), object(2)
memory usage: 276.2+ KB


In [37]:
dataf2.get_values

<bound method NDFrame.get_values of                        country                        attribute    year  \
0                      Austria                 Total population  1962.0   
1                      Austria                 Total population  1967.0   
2                      Austria                 Total population  1972.0   
3                      Austria                 Total population  1977.0   
4                      Austria                 Total population  1982.0   
...                        ...                              ...     ...   
7066  United States of America  Environmental Flow Requirements  1997.0   
7067  United States of America  Environmental Flow Requirements  2002.0   
7068  United States of America  Environmental Flow Requirements  2007.0   
7069  United States of America  Environmental Flow Requirements  2012.0   
7070  United States of America  Environmental Flow Requirements  2017.0   

       value  
0     7157.0  
1     7397.0  
2     7581.0  
3  

In [18]:
datac1.iloc[866]  # a partir de l'index 867 ce ne sont plus que des infos inutiles sur le fichier lui-même

country                            United States of America
attribute    Environmental Flow Requirements (10^9 m3/year)
year1                                                  1962
1958-1962                                            1491.0
year2                                                  1967
1963-1967                                            1491.0
year3                                                  1972
1968-1972                                            1491.0
year4                                                  1977
1973-1977                                            1491.0
year5                                                  1982
1978-1982                                            1491.0
year6                                                  1987
1983-1987                                            1491.0
year7                                                  1992
1988-1992                                            1491.0
year8                                   

In [19]:
datac2 = datac1[datac1.index < 867]
datac2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 867 entries, 0 to 866
Data columns (total 26 columns):
country      837 non-null object
attribute    837 non-null object
year1        462 non-null float64
1958-1962    492 non-null object
year2        462 non-null float64
1963-1967    492 non-null object
year3        503 non-null float64
1968-1972    533 non-null object
year4        505 non-null float64
1973-1977    535 non-null object
year5        535 non-null float64
1978-1982    565 non-null object
year6        535 non-null float64
1983-1987    565 non-null object
year7        653 non-null float64
1988-1992    683 non-null object
year8        649 non-null float64
1993-1997    679 non-null object
year9        699 non-null float64
1998-2002    729 non-null object
year10       682 non-null float64
2003-2007    712 non-null object
year11       718 non-null float64
2008-2012    748 non-null object
year12       668 non-null float64
2013-2017    698 non-null object
dtypes: float64(12), obje

In [35]:
datac2.loc[datac2['attribute'] == 'Total population (1000 inhab)']

,country,attribute,year1,1958-1962,year2,1963-1967,year3,1968-1972,year4,1973-1977,year5,1978-1982,year6,1983-1987,year7,1988-1992,year8,1993-1997,year9,1998-2002,year10,2003-2007,year11,2008-2012,year12,2013-2017
0,Austria,Total population (1000 inhab),1962.0,7157.0,1967.0,7397.0,1972.0,7581.0,1977.0,7637.0,1982.0,7602.0,1987.0,7639.0,1992.0,7831.0,1997.0,8033.0,2002.0,8134.0,2007.0,8312.0,2012.0,8518.0,2017.0,8735.0
28,Belgium,Total population (1000 inhab),1962.0,9193.0,1967.0,9476.0,1972.0,9651.0,1977.0,9752.0,1982.0,9821.0,1987.0,9868.0,1992.0,10016.0,1997.0,10193.0,2002.0,10365.0,2007.0,10698.0,2012.0,11084.0,2017.0,11429.0
56,Bosnia and Herzegovina,Total population (1000 inhab),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1992.0,4239.0,1997.0,3752.0,2002.0,3776.0,2007.0,3774.0,2012.0,3648.0,2017.0,3507.0
84,Bulgaria,Total population (1000 inhab),1962.0,8022.0,1967.0,8343.0,1972.0,8609.0,1977.0,8802.0,1982.0,8931.0,1987.0,8960.0,1992.0,8676.0,1997.0,8212.0,2002.0,7866.0,2007.0,7567.0,2012.0,7310.0,2017.0,7085.0
112,Cambodia,Total population (1000 inhab),1962.0,6029.0,1967.0,6685.0,1972.0,7300.0,1977.0,7194.0,1982.0,6918.0,1987.0,8196.0,1992.0,9622.0,1997.0,11296.0,2002.0,12635.0,2007.0,13677.0,2012.0,14777.0,2017.0,16005.0
140,Croatia,Total population (1000 inhab),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1992.0,4733.0,1997.0,4532.0,2002.0,4396.0,2007.0,4362.0,2012.0,4297.0,2017.0,4189.0
168,Denmark,Total population (1000 inhab),1962.0,4649.0,1967.0,4832.0,1972.0,4991.0,1977.0,5097.0,1982.0,5123.0,1987.0,41047.0,1992.0,5172.0,1997.0,5278.0,2002.0,5373.0,2007.0,5470.0,2012.0,5611.0,2017.0,5734.0
196,Finland,Total population (1000 inhab),1962.0,4499.0,1967.0,4589.0,1972.0,4651.0,1977.0,4747.0,1982.0,4835.0,1987.0,4946.0,1992.0,5045.0,1997.0,5152.0,2002.0,5213.0,2007.0,5298.0,2012.0,5413.0,2017.0,5523.0
224,France,Total population (1000 inhab),1962.0,47121.0,1967.0,49792.0,1972.0,51740.0,1977.0,53516.0,1982.0,54548.0,1987.0,56025.0,1992.0,57497.0,1997.0,58763.0,2002.0,60225.0,2007.0,61961.0,2012.0,63640.0,2017.0,64980.0
252,Germany,Total population (1000 inhab),1962.0,74489.0,1967.0,77353.0,1972.0,78902.0,1977.0,78702.0,1982.0,77945.0,1987.0,78054.0,1992.0,80046.0,1997.0,81500.0,2002.0,81612.0,2007.0,81344.0,2012.0,81066.0,2017.0,82114.0


In [25]:
datac2["country"].isna().sum()

30

In [24]:
datac2.isnull().sum()

country       30
attribute     30
year1        405
1958-1962    375
year2        405
1963-1967    375
year3        364
1968-1972    334
year4        362
1973-1977    332
year5        332
1978-1982    302
year6        332
1983-1987    302
year7        214
1988-1992    184
year8        218
1993-1997    188
year9        168
1998-2002    138
year10       185
2003-2007    155
year11       149
2008-2012    119
year12       199
2013-2017    169
dtype: int64

In [27]:
test = datac2[datac2.isna().any(axis=1)]
test

,country,attribute,year1,1958-1962,year2,1963-1967,year3,1968-1972,year4,1973-1977,year5,1978-1982,year6,1983-1987,year7,1988-1992,year8,1993-1997,year9,1998-2002,year10,2003-2007,year11,2008-2012,year12,2013-2017
6,Austria,Human Development Index (HDI) [highest = 1] (-),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990.0,0.7935,NaN,NaN,2000.0,0.8364,NaN,NaN,2012.0,0.8839,2014.0,0.885
18,Austria,Total exploitable water resources (10^9 m3/year),1962.0,93.0,1967.0,93.0,1972.0,93.0,1977.0,93.0,1982.0,93.0,1987.0,93.0,1992.0,93.0,1997.0,93.0,2002.0,93.0,2007.0,93.0,2012.0,93.0,NaN,NaN
21,Austria,Agricultural water withdrawal (10^9 m3/year),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980.0,0.09,1985.0,0.2,1990.0,0.1,1997.0,0.1,2002.0,0.1,NaN,NaN,2010.0,0.0771,NaN,NaN
22,Austria,Industrial water withdrawal (10^9 m3/year),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980.0,2.694,1985.0,2.671,1990.0,3.022,1997.0,2.857,1999.0,2.899,NaN,NaN,2008.0,2.695,NaN,NaN
23,Austria,Municipal water withdrawal (10^9 m3/year),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980.0,0.5582,1985.0,0.7088,1990.0,0.6851,1997.0,0.6871,1999.0,0.6687,NaN,NaN,2010.0,0.72,NaN,NaN
24,Austria,Total water withdrawal (10^9 m3/year),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980.0,3.342,1985.0,3.58,1990.0,3.807,1997.0,3.644,2002.0,3.668,NaN,NaN,2010.0,3.492,NaN,NaN
25,Austria,Total water withdrawal per capita (m3/inhab/year),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980.0,439.6,1985.0,468.6,1990.0,486.1,1997.0,453.6,2002.0,450.9,NaN,NaN,2010.0,410.0,NaN,NaN
27,NaN,NaN,NaN,1958-1962,NaN,1963-1967,NaN,1968-1972,NaN,1973-1977,NaN,1978-1982,NaN,1983-1987,NaN,1988-1992,NaN,1993-1997,NaN,1998-2002,NaN,2003-2007,NaN,2008-2012,NaN,2013-2017
31,Belgium,Population density (inhab/km2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002.0,339.5,2007.0,350.4,2012.0,363.1,2017.0,374.4
34,Belgium,Human Development Index (HDI) [highest = 1] (-),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990.0,0.8064,NaN,NaN,2000.0,0.8738,NaN,NaN,2012.0,0.8886,2014.0,0.8903


In [29]:
dataf2.head()

,country,attribute,year,value
0,Austria,Total population,1962.0,7157.0
1,Austria,Total population,1967.0,7397.0
2,Austria,Total population,1972.0,7581.0
3,Austria,Total population,1977.0,7637.0
4,Austria,Total population,1982.0,7602.0


26/08 - Point VH-PM : 
-  TERMINER DE NETTOYER DATAC2
- Supprimer les colonnes "year"
- REMPLACER LES NaN par des valeurs nulles (0)

#### dataf2 is already included in datac2

### Il faut maintenant concaténer les deux fichiers datac2 + scrapping de Zhifeng

## 3. Data Features - first on one country or region, then extension
- Datas from 1988 to 2019, year by year - idealy week per week !
- As many datafiles as Country
- Each file is a feature / date datafile
- Global water at disposal for population (by Country)
- Detail water at disposal (eventualy) : rivers (instant stream), dams (volume), underground (volume), etc.
- Population count (by country)
- GDP (gross domestic product or PIB or HDI human development index)
- Usage related datas :
- - average water consumption per inhabitant (by country)
- - average energy consumption per inhabitant (by country) or water consumption for energy production
- - eventualy, industrialisation rate or water use for industrie (by country)
- - eventualy, agricultural rate or water use for agriculture (by country)

## 4. Preprocessing
- data formating : for exemple 1 dataset for 1 country with the year as column and features as rows
- data cleaning
- FAO EDA simple analysis (Exploratory Data Analysis) :
- - https://www.kaggle.com/marcinjadwizyc/fao-simple-eda
- - https://www.kaggle.com/hungnd11/exploring-the-fao-dataset
- Hydrofunctions
- - https://github.com/mroberge/hydrofunctions

## 5. Modeling
- Statistics using Machine Learning
- Prediction with time series or others

## 6. Restitution
- Précontainérisation : fichier de variables de configuration multi-utilisateurs
- - Requirement.txt
- - launching file (an url file)
- Map and curves restitution on actual and predictive water stress by country
- Interactive web restitution